# Workshop Audio Cognitive

# 🤖 Atendimento Digital por Voz – QuantumFinance

Este projeto simula um atendimento digital da empresa fictícia **Quantum Finance**, permitindo que o cliente interaja **por voz** em vez de apenas por texto.

---

## 🔧 O que o sistema faz?

1. Dá boas-vindas ao cliente com uma mensagem em áudio.
2. Apresenta as seguintes opções:
   - (1) Consultar saldo da conta
   - (2) Simulação de compra internacional
   - (3) Falar com um atendente
   - (4) Sair do atendimento
3. O cliente fala sua escolha, e o sistema **reconhece a fala**.
4. A resposta correspondente é **reproduzida em áudio**.
5. Se não entender a opção, o sistema avisa e repete as opções.
6. O atendimento termina ao dizer "sair do atendimento".

---

## 📦 Tecnologias usadas

- `pyttsx3`: Geração de áudio com TTS (Text-to-Speech)
- `SpeechRecognition`: Reconhecimento de voz com SST

---



In [2]:
# Primeiro nós importamos todas as ferramentas que vamos usar
import pyttsx3  # faz o computador falar - TTS (Text-to-Speech: transforma texto em voz)
import speech_recognition as sr  # faz o computador ouvir e entender nossa voz (STT - Speech-to-Text: converte voz em texto)
import pygame  # ajuda a tocar sons
import time  # para fazer pausas e esperar
import os  # para trabalhar com pastas e arquivos
from difflib import SequenceMatcher  # para comparar palavras parecidas

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# 🎛️ Configurações
pygame.mixer.init()  # Prepara o módulo de som
engine = pyttsx3.init()  # Prepara o motor de fala
engine.setProperty('rate', 160)  # Ajusta a velocidade da voz (160 palavras por minuto)

In [4]:
# 📏 Quão parecidas as palavras precisam ser para o computador entender
SIMILARITY_THRESHOLD = 0.6  # 60% de similaridade já é suficiente

# 📂 Cria uma pasta chamada "audios" SE ela não existir
os.makedirs("audios", exist_ok=True)

# 💬 Todas as frases que o sistema vai falar
frases = {
    "saudacao": "Bem-vindo ao atendimento da Quantum Finance. Por favor fale com clareza.",   
    "opcoes": "Opções disponíveis: 1-Saldo, 2-Simulação, 3-Atendente, 4-Sair.",
    "opcao1": "Você escolheu consultar saldo.",
    "opcao2": "Você escolheu simulação internacional.",
    "opcao3": "Você escolheu falar com atendente.",
    "opcao4": "Encerrando atendimento. Obrigado!",
    "erro": "Não entendi. Por favor repita."
}

# 🔊 Função que cria arquivos de áudio para cada frase
def gerar_audios():
    # Verifica se os áudios já existem
    for nome, texto in frases.items():  # Para cada frase...
        caminho = os.path.join("audios", f"{nome}.wav")  # Cria o caminho do arquivo
        if not os.path.exists(caminho):  # Se o áudio ainda não existe...
            engine.save_to_file(texto, caminho)  # Grava a frase em um arquivo
    engine.runAndWait()  # Espera terminar de gravar todos

In [5]:
# ▶️ Função que toca um áudio
def play_audio(nome):
    try:
        caminho = os.path.join("audios", f"{nome}.wav")  # Encontra o arquivo
        if os.path.exists(caminho):  # Se o arquivo existe...
            pygame.mixer.music.load(caminho)  # Carrega o áudio
            pygame.mixer.music.play()  # Toca o áudio
            while pygame.mixer.music.get_busy():  # Enquanto estiver tocando...
                time.sleep(0.1)  # Espera um pouquinho (0.1 segundo)
    except Exception as e:  # Se der erro...
        print(f"Erro de áudio: {e}")  # Mostra o que aconteceu

In [6]:
import speech_recognition as sr

# 👂 Função que escuta o que a pessoa fala
def ouvir_audio():
    r = sr.Recognizer()  # Cria o objeto que reconhece voz
    
    with sr.Microphone() as source:  # Abre o microfone
        print("\n🔊 Pronto para ouvir...")
        
        # Ajusta para ruído ambiente (1 segundo de análise)
        r.adjust_for_ambient_noise(source, duration=1)
        
        # Configurações de pausa (0.8 segundos de silêncio = fim de fala)
        r.pause_threshold = 0.8
        
        try:
            # Toca o áudio de instrução
            play_audio("instrucao")  # Toca a instrução para o usuário 
            
            # Escuta o áudio com timeout (7s total, máximo 4s de fala)
            audio = r.listen(source, timeout=7, phrase_time_limit=4)
            
            try:
                # reconhecimento dentro de outro try para capturar erros específicos
                texto = r.recognize_google(audio_data=audio, language="pt-BR").lower()
                print(f"👉 Você disse: {texto}")
                return texto
                
            except sr.UnknownValueError:
                print("🔇 Não consegui entender o áudio")
                return None
            except sr.RequestError as e:
                print(f"🌐 Erro na API: {e}")
                return None
                
        except sr.WaitTimeoutError:
            print("⏳ Nenhuma fala detectada (timeout)")
            return None
        except Exception as e:
            print(f"❌ Erro inesperado: {e}")
            return None

In [7]:
# 🔍 Função que descobre qual opção a pessoa escolheu
def identificar_opcao(fala):
    # Todas as palavras que representam cada opção
    opcoes = {
        1: ["saldo", "conta", "1", "um", "extrato", "consultar saldo", "consultar extrato", "ver saldo", "ver extrato"],
        2: ["simulação", "simular", "2", "dois", "internacional", "simular compra internacional", "simular compra"],
        3: ["atendente", "atendimento", "3", "três", "humano", "falar com atendente", "falar com humano", "falar com atendente humano", "falar com", "falar"],
        4: ["sair", "encerrar", "4", "quatro", "fim", "desligar", "fechar", "terminar", "sair do atendimento", "sair do sistema", "encerrar atendimento", "encerrar sistema"],
    }
    
    if not fala:  # Se não ouviu nada...
        return 0
    
    # Compara o que a pessoa falou com todas as opções
    for num, palavras in opcoes.items():  # Para cada opção...
        for palavra in palavras:  # Para cada palavra-chave...
            # Se a palavra está no que foi dito OU é parecida o suficiente
            if palavra in fala or SequenceMatcher(None, fala, palavra).ratio() >= 0.6:
                return num  # Retorna o número da opção
    return 0  # Se não encontrou nada

In [8]:
# 🏦 Função principal que realiza o atendimento
def atendimento():
    gerar_audios()  # Cria todos os áudios primeiro
    play_audio("saudacao")  # Dá as boas-vindas
    
    while True:  # Repete para sempre até sair
        play_audio("opcoes")  # Fala as opções
        fala = ouvir_audio()  # Escuta a resposta
        opcao = identificar_opcao(fala)  # Tenta entender o que foi dito
        
        if opcao == 0:  # Se não entendeu...
            play_audio("erro")  # Pede para repetir
            continue  # Volta para o início
            
        print(f"\n✅ Reconhecido: Opção {opcao}")  # Mostra a opção entendida
        play_audio(f"opcao{opcao}")  # Fala a confirmação
        
        if opcao == 4:  # Se escolheu sair...
            print("Atendimento finalizado!")
            break  # Termina o atendimento

# ⚡ Executa o programa
if __name__ == "__main__":
    print("ATENDIMENTO QUANTUM FINANCE")
    atendimento()  # Inicia o atendimento

ATENDIMENTO QUANTUM FINANCE

🔊 Pronto para ouvir...
👉 Você disse: consultar saldo

✅ Reconhecido: Opção 1

🔊 Pronto para ouvir...
👉 Você disse: simular compra internacional

✅ Reconhecido: Opção 2

🔊 Pronto para ouvir...
👉 Você disse: tomar banho

🔊 Pronto para ouvir...
👉 Você disse: falar com atende

✅ Reconhecido: Opção 3

🔊 Pronto para ouvir...
👉 Você disse: encerrar

✅ Reconhecido: Opção 4
Atendimento finalizado!
